In [12]:
using JuMP, HiGHS, Ipopt

function variableref(args...)
    return Array{JuMP.VariableRef, length(args)}(undef, args...)
end
function constraintref(args...)
    return Array{JuMP.ConstraintRef, length(args)}(undef, args...)
end
function expressionref(args...)
    return Array{JuMP.AffExpr, length(args)}(undef, args...)
end

Base.@kwdef mutable struct DCOPFModel
    optimizer::JuMP.Model = Model()
    var::Dict{String, Any} = Dict{String, Any}()
    con::Dict{String, Any} = Dict{String, Any}()
    expr::Dict{String, Any} = Dict{String, Any}()
    obj_exp::JuMP.AffExpr = zero(JuMP.AffExpr)
end

Base.@kwdef mutable struct DCOPFInputs
    phase_reference::Int = 1
    Ybus::Matrix{Complex{Float64}} = Complex{Float64}[;;]
    min_flow::Matrix{Float64} = Float64[;;]
    max_flow::Matrix{Float64} = Float64[;;]
    min_generation::Vector{Float64} = Float64[]
    max_generation::Vector{Float64} = Float64[]
    generation_cost::Vector{Float64} = Float64[]
    demand::Vector{Float64} = Float64[]
    consider_losses::Bool = false
    max_iterations::Int = 1
    tolerance::Float64 = 1e-4
end

Base.@kwdef mutable struct DCOPFResults
    # tuple is name and iteration
    var::Dict{Tuple{String, Float64}, Any} = Dict{Tuple{String, Float64}, Any}()
    expr::Dict{Tuple{String, Float64}, Any} = Dict{Tuple{String, Float64}, Any}()
    obj_exp::JuMP.AffExpr = zero(JuMP.AffExpr)
end

function linear_flow!(model::DCOPFModel, inputs::DCOPFInputs)
    optimizer_model = model.optimizer
    
    Ybus = inputs.Ybus
    Gbus = real(Ybus)
    Xbus = imag(Ybus.^(-1))
    num_bus = size(Ybus)[1]
    
    phase_reference = inputs.phase_reference
    demand = inputs.demand
    min_flow = inputs.min_flow
    max_flow = inputs.max_flow
    max_generation = inputs.max_generation
    min_generation = inputs.min_generation
    
    if !inputs.consider_losses
        power_loss = zeros(AffExpr, num_bus)
    else
        power_loss = model.expr["PowerLossPerBus"]
    end
    
    ### create variables
    phase = variableref(num_bus)
    generation = variableref(num_bus)
    
    ### create constraints
    load_balance = constraintref(num_bus)
    flow_lower_bound = constraintref(num_bus, num_bus)
    flow_upper_bound = constraintref(num_bus, num_bus)
    
    ### create expressionss
    flow = zeros(AffExpr, num_bus, num_bus)
    power_injection = expressionref(num_bus)
    
    ### define variables
    for bus in 1:num_bus
        generation[bus] = @variable(
            optimizer_model,
            lower_bound = min_generation[bus],
            upper_bound = max_generation[bus]
        )
        if bus == phase_reference
            phase[bus] = @variable(
                optimizer_model,
                lower_bound = 0,
                upper_bound = 0
            )
        else
            phase[bus] = @variable(
                optimizer_model
            )
        end
    end
    
    ### define expressions
    for bus_from in 1:num_bus, bus_to in 1:num_bus
        if bus_from == bus_to
            continue
        end
        flow[bus_from, bus_to] = @expression(
            optimizer_model,
            (phase[bus_from] - phase[bus_to])/Xbus[bus_from, bus_to]
        )
    end
    for bus in 1:num_bus
        power_injection[bus] = @expression(
            optimizer_model,
            sum(flow[bus, :])    
        )
    end
    
    ### define constraints
    for bus in 1:num_bus
        load_balance[bus] = @constraint(
            optimizer_model,
            generation[bus] == demand[bus] + power_injection[bus] + power_loss[bus]
        )
    end
    for bus_from in 1:num_bus, bus_to in 1:num_bus
        flow_upper_bound[bus_from, bus_to] = @constraint(
            optimizer_model,
            flow[bus_from, bus_to] <= max_flow[bus_from, bus_to]
        )
        flow_lower_bound[bus_from, bus_to] = @constraint(
            optimizer_model,
            flow[bus_from, bus_to] >= min_flow[bus_from, bus_to]
        )
    end
    
    ### save variables
    model.var["Phase"] = phase
    model.var["Generation"] = generation
    
    ### save constraints
    model.con["LoadBalance"] = load_balance
    model.con["FlowLowerBound"] = flow_lower_bound
    model.con["FlowUpperBound"] = flow_upper_bound
    
    ### save expressions
    model.expr["Flow"] = flow
    model.expr["PowerInjection"] = power_injection
end

function begin_loss_cuts!(model::DCOPFModel, input::DCOPFInputs)
    Ybus = inputs.Ybus
    num_bus = size(Ybus)[1]
    
    for bus_from in 1:num_bus, bus_to in 1:num_bus
        
    end
end

function loss_cuts!(model::DCOPFModel, input::DCOPFInputs)
end

function new_loss_cut!(model::DCOPFModel, input::DCOPFInputs, results::DCOPFResults, current_iteration::Int)
    Ybus = inputs.Ybus
    num_bus = size(Ybus)[1]

    phase = results.var["Phase", current_iteration]
    loss_cut = Array{Vector{Float64}, 2}(undef, num_bus, num_bus)

    for bus_from in 1:num_bus, bus_to in 1:num_bus
        phase_difference = phase[bus_from] - phase[bus_to]
        conductance = real(Ybus)[bus_from, bus_to]
        loss_derivative = transmission_loss_derivative(phase_difference, conductance)
        loss = transmission_losss(phase_difference, conductance)
        # cut is a vector [a, b] where y = a*x + b
        loss_cut[bus_from, bus_to] = [loss_derivative, loss - (loss_derivative * phase_difference)]
    end
end

function save_all_results!(model::DCOPFModel, results::DCOPFResults, current_iteration::Int)
    for (key, group) in model.var
        results.var[key, current_iteration] = JuMP.value.(group)
    end
    for (key, group) in model.expr
        results.expr[key, current_iteration] = JuMP.value.(group)
    end
    return nothing
end
    
function transmission_loss(phase_difference::Float64, conductance::Float64)
    return conductance * phase_difference^2
end

function transmission_loss_derivative(phase_difference::Float64, conductance::Float64)
    return 2 * conductance * phase_difference
end

transmission_loss_derivative (generic function with 1 method)

In [13]:
model = DCOPFModel()
model.optimizer = Model(HiGHS.Optimizer)

inputs = DCOPFInputs()
inputs.Ybus = [ 22.5im -10im -12.5im; -10im 30im -20im; -12.5im -20im 32.5im ]
inputs.demand = [0, 0, .8]
inputs.min_generation = zeros(3)
inputs.max_generation = [.5, .5, 0]
max_flow = [0 .1 .5; .1 0 .5; .5 .5 0]
inputs.min_flow = -max_flow
inputs.max_flow = max_flow
cost = [.5, 1, 0]
inputs.generation_cost = cost

linear_flow!(model, inputs)
generation = model.var["Generation"]
@objective(model.optimizer, Min, sum(cost .* generation))
optimize!(model.optimizer)

results = DCOPFResults()
save_all_results!(model, results, 1)

Running HiGHS 1.5.1 [date: 1970-01-01, git hash: 93f1876e4]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11 rows, 4 cols, 20 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-21); columns 0(-6); elements 0(-36) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.5769230769e-01
HiGHS run time      :          0.00


In [15]:
results.var["Generation", 1] |> println
results.expr["Flow", 1] |> display

[0.48461538461538467, 0.31538461538461543, 0.0]


3×3 Matrix{Float64}:
  0.0        0.1       0.384615
 -0.1        0.0       0.415385
 -0.384615  -0.415385  0.0